In [1]:
# gpu number setting
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '1' ## gpu 번호 셋팅 윤건 :0, 기준 : 1, 준형 :2,

# Gpu growth setting
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)


# tensorflow & keras version check
print('tensorflow version : ' , tf.__version__)
print('keras version : ' , tf.keras.__version__)

# tensorflow gpu available check 
print('GPU available ? : ', tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

tensorflow version :  2.3.1
keras version :  2.4.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU available ? :  True


In [2]:
import pandas as pd
import numpy as np
import pickle
import tqdm
import random
import time

In [3]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [4]:
with open('../../Data/movie_201118_table_5.pickle', 'rb') as f:
    raw = pickle.load(f)

In [5]:
raw

,index,sa_id,album_id,release_date,run_time,meta_genre,genre_large,genre_mid,genre_small,movie_meta_price,buy_history_price,i30_meta_price,buy_tot,buy_seg,amt_1_4,amt_2_4,amt_3_4,amt_4_4,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
0,0,205101491713,M01208D595PPV00,20200715,115,액션,영화,액션,N,9900,\N,9900,0,VIEW_NO,1.0,0.000,0.0,0.000,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
1,1,205101521683,M011526134PPV00,20050714,85,극장판 애니,영화,극장판 애니,어린이/가족,1400,\N,1400,0,FOD,1.0,0.000,0.0,0.000,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False
2,2,205101521683,M01208A459PPV00,20110113,95,극장판 애니,영화,극장판 애니,어린이/가족,1400,\N,1400,0,FOD,1.0,0.000,0.0,0.000,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False
3,3,205101521683,M01188R075PPV00,20171215,112,액션,영화,액션,N,1400,\N,1400,0,FOD,1.0,0.000,0.0,0.000,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
4,4,205101521683,M01133TA30PPV00,20121129,97,극장판 애니,영화,극장판 애니,어린이/가족,1400,\N,1400,0,FOD,1.0,0.000,0.0,0.000,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8459259,8459359,M18061900001,M012054529PPV00,20120606,94,극장판 애니,영화,극장판 애니,어린이/가족,13900,\N,13900,8,BUY_3,0.0,0.375,0.0,0.625,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False
8459260,8459360,M18061900001,M01204M075PPV00,20190925,120,스릴러,영화,스릴러,N,3500,3500,3500,8,BUY_3,0.0,0.375,0.0,0.625,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
8459261,8459361,M18061900001,M01206U082PPV00,20200610,113,코미디,영화,코미디,N,7700,9900,7700,8,BUY_3,0.0,0.375,0.0,0.625,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8459262,8459362,M18061900001,M01206U082PPV00,20200610,113,코미디,영화,코미디,N,7700,9900,7700,8,BUY_3,0.0,0.375,0.0,0.625,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


* final drop

In [6]:
num_genre = 33

In [7]:
sparse_features = ['sa_id', 'album_id', 'buy_seg'] + [str(i) for i in range(num_genre)]
dense_features = ['release_date', 'run_time', 'movie_meta_price', 'i30_meta_price', 'buy_tot',
                  'amt_1_4', 'amt_2_4', 'amt_3_4', 'amt_4_4']
unnecessary_features = ['index', 'meta_genre', 'genre_large', 'genre_mid', 'genre_small', 'buy_history_price']

In [8]:
print(len(raw.columns))
print(len(sparse_features))
print(len(dense_features))
print(len(unnecessary_features))

51
36
9
6


In [9]:
raw.drop(columns=unnecessary_features, inplace=True)

In [14]:
raw['label'] = 1

In [15]:
raw

,sa_id,album_id,release_date,run_time,movie_meta_price,i30_meta_price,buy_tot,buy_seg,amt_1_4,amt_2_4,amt_3_4,amt_4_4,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,label
0,205101491713,M01208D595PPV00,20200715,115,9900,9900,0,VIEW_NO,1.0,0.000,0.0,0.000,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,1
1,205101521683,M011526134PPV00,20050714,85,1400,1400,0,FOD,1.0,0.000,0.0,0.000,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,1
2,205101521683,M01208A459PPV00,20110113,95,1400,1400,0,FOD,1.0,0.000,0.0,0.000,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,1
3,205101521683,M01188R075PPV00,20171215,112,1400,1400,0,FOD,1.0,0.000,0.0,0.000,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,1
4,205101521683,M01133TA30PPV00,20121129,97,1400,1400,0,FOD,1.0,0.000,0.0,0.000,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8459259,M18061900001,M012054529PPV00,20120606,94,13900,13900,8,BUY_3,0.0,0.375,0.0,0.625,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,1
8459260,M18061900001,M01204M075PPV00,20190925,120,3500,3500,8,BUY_3,0.0,0.375,0.0,0.625,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,1
8459261,M18061900001,M01206U082PPV00,20200610,113,7700,7700,8,BUY_3,0.0,0.375,0.0,0.625,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,1
8459262,M18061900001,M01206U082PPV00,20200610,113,7700,7700,8,BUY_3,0.0,0.375,0.0,0.625,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,1


In [16]:
tmp = time.time()
data_dict = raw.set_index(keys=['sa_id', 'album_id']).T.to_dict('list')
print(time.time() - tmp)

/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  


981.6980028152466


In [18]:
user_col = ['sa_id', 'buy_seg', 'buy_tot']
cont_col = ['album_id', 'release_date', 'run_time', 'movie_meta_price', 'i30_meta_price',
                'amt_1_4', 'amt_2_4', 'amt_3_4', 'amt_4_4'] + [str(i) for i in range(num_genre)]

* all data -> user_table & content_table

In [19]:
user_tab = raw[user_col]
cont_tab = raw[cont_col]

* duplicate rows 제거

In [23]:
user_uni_tab = user_tab.loc[list(user_tab['sa_id'].drop_duplicates().index)]
cont_uni_tab = cont_tab.loc[list(cont_tab['album_id'].drop_duplicates().index)]

In [10]:
from collections import defaultdict

In [18]:
counter = defaultdict(list)

* original method

In [11]:
def get_train_instances(train, num_negatives, num_items):
    '''
    설명 : train matrix를 받아서 그 안에 있는 user, item의 조합 1개당 num_negatives의 수만큼 negative sample을 만드는 함수.
    '''
#     user_input, item_input, labels = [],[],[]
    output_dict = defaultdict(list)
    
    train_keys = train.keys()
    train_items = train.items()
    for (u, i) in tqdm.tqdm(train_keys):       
        # positive instance
        for feat in user_col:
            
            user_input.append(u)
            item_input.append(i)
            labels.append(1)
#             output_dict[(u, i)] = 1
        # negative instances
        for _ in range(num_negatives):
            j = random.randint(0,num_items-1)
            while (u,j) in train_keys :
                j = random.randint(0, num_items-1)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
#             output_dict[(u, j)] = 0
            
        
    return final_dict, user_input, item_input, labels

In [9]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from deepctr.models import *
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names

In [11]:
for feat in sparse_features:
    lbe = LabelEncoder()
    data_2[feat] = lbe.fit_transform(data_2[feat])
    
mms = MinMaxScaler(feature_range=(0, 1))
data_2[dense_features] = mms.fit_transform(data_2[dense_features])

/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying t

In [17]:
final_dict, user_input, item_input, labels = get_train_instances(data_dic, 1, max(data_2['album_id']))

100%|██████████| 4623726/4623726 [00:19<00:00, 238378.32it/s]


In [21]:
tmp = time.time()
user_uni_dic = user_uni_tab.set_index(keys=['sa_id']).T.to_dict('list')
cont_uni_dic = cont_uni_tab.set_index(keys=['album_id']).T.to_dict('list')
print(time.time() - tmp)

58.737810134887695


In [30]:
user_input = final_dict[1]
item_input = final_dict[2]
labels = final_dict[3]

In [32]:
final_dict = final_dict[0]

In [100]:
final_df = pd.DataFrame({'sa_id': user_input, 'album_id':item_input, 'label':labels})

In [101]:
final_df

,sa_id,album_id,label
0,68,8282,1
1,68,7954,0
2,103,1724,1
3,103,7390,0
4,103,2542,1
...,...,...,...
9247447,968006,5754,0
9247448,968006,10376,1
9247449,968006,886,0
9247450,968006,10271,1


In [75]:
user_col.remove('sa_id')
cont_col.remove('album_id')

In [99]:
print(user_col)

['buy_seg', 'buy_tot']


## 아그냥 pandas column 추가해서 column마다 apply 가자

In [82]:
for idx, row in final_df.iterrows():
    print(idx)
    print(row)
    sa_id = row[0]
    album_id = row[1]
    user_values = user_uni_dic[sa_id]
    cont_values = cont_uni_dic[album_id]
    print(user_values)
    print(cont_values)
#     break
    for feat in user_col:
        test_df.assign(feat=lambda x: user_uni_dic[x['sa_id'].values[0]])
    break
    

0
sa_id         68
album_id    8282
label          1
Name: 0, dtype: int64
[3.0, 0.0]
[0.9878830984094158, 0.48010610079575594, 0.05102040816326531, 0.05102040816326531, 0.05102040816326531, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]


ValueError: Length of values does not match length of index

In [96]:
for i in tqdm.tqdm(range(len(final_df))):
    sa_id = final_df.loc[i, 'sa_id']
    album_id = final_df.loc[i, 'album_id']
    
    for j in range(len(user_col)):
        final_df.loc[i, user_col[j]] = user_uni_dic[sa_id][j]
                   
#     for j in range(5):
#         final_df.loc[i, cont_col[j]] = cont_uni_dic[album_id][j]
        





  0%|          | 0/9247452 [00:00<?, ?it/s]



  0%|          | 2/9247452 [00:00<139:20:20, 18.44it/s]



  0%|          | 4/9247452 [00:00<139:50:20, 18.37it/s]



  0%|          | 6/9247452 [00:00<140:12:07, 18.32it/s]



  0%|          | 8/9247452 [00:00<140:31:17, 18.28it/s]



  0%|          | 10/9247452 [00:00<140:40:10, 18.26it/s]



  0%|          | 12/9247452 [00:00<140:47:51, 18.24it/s]



  0%|          | 14/9247452 [00:00<140:51:18, 18.24it/s]



  0%|          | 16/9247452 [00:00<140:52:12, 18.23it/s]



  0%|          | 18/9247452 [00:00<140:55:29, 18.23it/s]



  0%|          | 20/9247452 [00:01<140:59:48, 18.22it/s]



  0%|          | 22/9247452 [00:01<140:59:12, 18.22it/s]



  0%|          | 24/9247452 [00:01<140:59:32, 18.22it/s]



  0%|          | 26/9247452 [00:01<140:57:47, 18.22it/s]



  0%|          | 28/9247452 [00:01<140:58:48, 18.22it/s]



  0%|          | 30/9247452 [00:01<140:57:51, 18.22it/s]



  0%|          | 32/9247452 [00:01<140:57:47, 18.22it

KeyboardInterrupt: 

In [102]:
final_df

,sa_id,album_id,label
0,68,8282,1
1,68,7954,0
2,103,1724,1
3,103,7390,0
4,103,2542,1
...,...,...,...
9247447,968006,5754,0
9247448,968006,10376,1
9247449,968006,886,0
9247450,968006,10271,1


In [103]:
user_uni_tab

,sa_id,buy_seg,buy_tot
0,68,3,0.000000
1,103,3,0.000000
4,114,0,0.003003
7,120,3,0.000000
8,121,2,0.000000
...,...,...,...
6759035,967907,1,0.009009
6759047,967935,1,0.012012
6759089,967966,1,0.018018
6759113,967993,3,0.000000


In [104]:
merge_df = pd.merge(final_df, user_uni_tab, how='inner', on='sa_id')

In [106]:
merge_df[:100]

,sa_id,album_id,label,buy_seg,buy_tot
0,68,8282,1,3,0.000000
1,68,7954,0,3,0.000000
2,103,1724,1,3,0.000000
3,103,7390,0,3,0.000000
4,103,2542,1,3,0.000000
5,103,9626,0,3,0.000000
6,103,257,1,3,0.000000
7,103,5377,0,3,0.000000
8,114,8512,1,0,0.003003
9,114,1097,0,0,0.003003


In [107]:
merge_df_2 = pd.merge(merge_df, cont_uni_tab, how='inner', on='album_id')

In [108]:
merge_df_2

,sa_id,album_id,label,buy_seg,buy_tot,release_date,run_time,movie_meta_price,i30_meta_price,buy_history_price,amt_1_4,amt_2_4,amt_3_4,amt_4_4,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,68,8282,1,3,0.000000,0.987883,0.480106,0.051020,0.051020,0.051020,1.0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,7064,8282,1,3,0.000000,0.987883,0.480106,0.051020,0.051020,0.051020,1.0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,7205,8282,1,3,0.000000,0.987883,0.480106,0.051020,0.051020,0.051020,1.0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,9056,8282,1,0,0.006006,0.987883,0.480106,0.051020,0.051020,0.051020,1.0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,9995,8282,0,2,0.000000,0.987883,0.480106,0.051020,0.051020,0.051020,1.0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9247447,889461,688,0,2,0.000000,0.812131,0.297082,0.028571,0.028571,0.028571,0.0,0.900000,0.100000,0.000000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9247448,900784,688,0,2,0.000000,0.812131,0.297082,0.028571,0.028571,0.028571,0.0,0.900000,0.100000,0.000000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9247449,359349,10569,1,1,1.000000,0.999999,0.238727,0.102041,0.102041,0.204082,0.0,0.222222,0.309309,0.468468,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9247450,531453,10569,1,1,0.966967,0.999999,0.238727,0.102041,0.102041,0.204082,0.0,0.222222,0.309309,0.468468,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [110]:
with open('../uplus_exp/raw_mod_4_negative_sample.pickle', 'wb') as f:
    pickle.dump(merge_df_2, f)